# Cancelamento de Clientes - Telco (dataset criado pela IBM para demonstração da ferramenta IBM Cognos Analytics)

### Contém informações sobre uma empresa fictícia de telecomunicações que forneceu serviços de telefonia residencial e internet para 7043 clientes na Califórnia no 3º trimestre.

### Etapa de preparação do dado - Realizado por Sabrina Otoni da Silva - 2024/01

### Objetivo: Preparar o dado para a etapa de visualização e exploração do dado.

In [1]:
from pathlib import Path

import pandas as pd 

from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

import sys
import os

automations_dir = os.path.join(os.getcwd(), '../automations')

if automations_dir not in sys.path:
    sys.path.append(automations_dir)

import data_processing as processing

In [2]:
datapath = Path('../data')
xlsx_path = Path(f'{datapath}/d01_raw/telco_customer_churn.xlsx')

In [3]:
data = pd.read_excel(f'{xlsx_path}' )
data.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [5]:
data.loc[data['CustomerID'].duplicated()]

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason


Não há mais de uma linha para cada CustomerID, então é uma coluna que não vai agregar nas análises e nos tratamentos.

In [6]:
l1 = [len(str(i).split()) for i in data['Total Charges']]
l2 = [i for i in range(len(l1)) if l1[i] != 1]
print('Posições dos índices com espaços vazios: ',*l2)

Posições dos índices com espaços vazios:  2234 2438 2568 2667 2856 4331 4687 5104 5719 6772 6840


Verificação realizada por conta de que, na tentativa de transformar a coluna Total Charges em float, a seguinte mensagem apareceu: "could not convert string to float: ' '".

In [7]:
data.iloc[l2, :]

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
2234,4472-LVYGI,1,United States,California,San Bernardino,92408,"34.084909, -117.258107",34.084909,-117.258107,Female,...,Two year,Yes,Bank transfer (automatic),52.55,,No,0,36,2578,NaN
2438,3115-CZMZD,1,United States,California,Independence,93526,"36.869584, -118.189241",36.869584,-118.189241,Male,...,Two year,No,Mailed check,20.25,,No,0,68,5504,NaN
2568,5709-LVOEQ,1,United States,California,San Mateo,94401,"37.590421, -122.306467",37.590421,-122.306467,Female,...,Two year,No,Mailed check,80.85,,No,0,45,2048,NaN
2667,4367-NUYAO,1,United States,California,Cupertino,95014,"37.306612, -122.080621",37.306612,-122.080621,Male,...,Two year,No,Mailed check,25.75,,No,0,48,4950,NaN
2856,1371-DWPAZ,1,United States,California,Redcrest,95569,"40.363446, -123.835041",40.363446,-123.835041,Female,...,Two year,No,Credit card (automatic),56.05,,No,0,30,4740,NaN
4331,7644-OMVMY,1,United States,California,Los Angeles,90029,"34.089953, -118.294824",34.089953,-118.294824,Male,...,Two year,No,Mailed check,19.85,,No,0,53,2019,NaN
4687,3213-VVOLG,1,United States,California,Sun City,92585,"33.739412, -117.173334",33.739412,-117.173334,Male,...,Two year,No,Mailed check,25.35,,No,0,49,2299,NaN
5104,2520-SGTTA,1,United States,California,Ben Lomond,95005,"37.078873, -122.090386",37.078873,-122.090386,Female,...,Two year,No,Mailed check,20.00,,No,0,27,3763,NaN
5719,2923-ARZLG,1,United States,California,La Verne,91750,"34.144703, -117.770299",34.144703,-117.770299,Male,...,One year,Yes,Mailed check,19.70,,No,0,69,4890,NaN
6772,4075-WKNIU,1,United States,California,Bell,90201,"33.970343, -118.171368",33.970343,-118.171368,Female,...,Two year,No,Mailed check,73.35,,No,0,44,2342,NaN


In [8]:
print('True') if (data.loc[data['Tenure Months'] == 0].index == l2).all() else print('False')

True


Para todos os Total Charges vazios, a coluna Tenure Months está zerada, mas Monthly Charges contém valores. 
Essa verificação foi feita pois, caso ocorresse de Tenure Months não estar zerada, eu poderia apenas multiplicar Tenure Months com Monthly Charges para obter o valor de Total Charges.

In [9]:
data.loc[(data['Monthly Charges'].astype(str) == data['Total Charges'].astype(str)) & data['Tenure Months'] != 0, 'Count'].count()

581

Como existem muitas ocasiões de Monthly Charges serem iguais a Total Charges, e a coluna Tenure Months estar zerada só para os casos de valor nulo em Total Charges, o tratamento será feito duplicando o valor de Monthly Charges para Total Charges.

In [10]:
unique_values= {col: data[col].unique() for col in data.columns}
for col, values in unique_values.items():
    print(f'Valores na coluna "{col}": {values}')
    print('--------------------------------------')

Valores na coluna "CustomerID": ['3668-QPYBK' '9237-HQITU' '9305-CDSKC' ... '2234-XADUH' '4801-JZAZL'
 '3186-AJIEK']
--------------------------------------
Valores na coluna "Count": [1]
--------------------------------------
Valores na coluna "Country": ['United States']
--------------------------------------
Valores na coluna "State": ['California']
--------------------------------------
Valores na coluna "City": ['Los Angeles' 'Beverly Hills' 'Huntington Park' ... 'Standish' 'Tulelake'
 'Olympic Valley']
--------------------------------------
Valores na coluna "Zip Code": [90003 90005 90006 ... 96128 96134 96146]
--------------------------------------
Valores na coluna "Lat Long": ['33.964131, -118.272783' '34.059281, -118.30742' '34.048013, -118.293953'
 ... '40.346634, -120.386422' '41.813521, -121.492666'
 '39.191797, -120.212401']
--------------------------------------
Valores na coluna "Latitude": [33.964131 34.059281 34.048013 ... 40.346634 41.813521 39.191797]
---------------

In [11]:
data['Churn Reason'].isna().sum()

5174

A coluna Churn Reason não será retirada, pelo menos por enquanto, mesmo havendo muitos valores nulos.

Serão deletadas as colunas: 
- CustomerID (não vai agregar nas análises);
- Count (só contêm um valor, sem agregar em nada); 
- Country (se trata de dados apenas dos Estados Unidos);
- State (por serem dados apenas da California);
- Zip Code (não contribui para as análises);
- Lat Long (já existe uma coluna de latitude e uma de longitude, facilitando as plotagens);
- Churn Label (já existe a coluna Churn Value que diz a mesma coisa e já esta em formato binário para a classificação)
- Churn Score (é uma feature preditiva (probabilidade), calculada pela ferramenta IBM SPSS Modeler, por razões de data leakege e enviesamento de possíveis resultados errados, não estarei alimentando meu futuro modelo com ela);
- CLTV (também é uma feature preditiva, calculada usando dados existentes e algumas fórmulas não mencionadas pelo autor dos dados, quanto maior o número, mais valor tem o cliente, vou retirar por precaução de data leakege e enviesamento do modelo, e o intuito do projeto é justamente criar um modelo que faça esse trabalho).

In [12]:
pipeline = Pipeline([
    ('import_drop', processing.DropColumns(drop_columns=['CustomerID', 'Count', 'Country', 'State', 'Zip Code', 
                                                                             'Lat Long', 'Churn Label', 'Churn Score', 'CLTV'])),
    ('data_treatment', processing.DataTreat())
])

In [13]:
data = pipeline.transform(data)
data.head()

True


,City,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,...,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value,Churn Reason
0,Los Angeles,33.964131,-118.272783,Male,No,No,No,2,Yes,No,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,Competitor made better offer
1,Los Angeles,34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,Moved
2,Los Angeles,34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,1,Moved
3,Los Angeles,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,...,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1,Moved
4,Los Angeles,34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,...,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,1,Competitor had better devices


In [14]:
data.iloc[l2, :]

,City,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,...,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value,Churn Reason
2234,San Bernardino,34.084909,-117.258107,Female,No,Yes,No,0,No,No phone service,...,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,52.55,0,NaN
2438,Independence,36.869584,-118.189241,Male,No,No,No,0,Yes,No,...,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,20.25,0,NaN
2568,San Mateo,37.590421,-122.306467,Female,No,Yes,No,0,Yes,No,...,No,Yes,Yes,Two year,No,Mailed check,80.85,80.85,0,NaN
2667,Cupertino,37.306612,-122.080621,Male,No,Yes,Yes,0,Yes,Yes,...,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,25.75,0,NaN
2856,Redcrest,40.363446,-123.835041,Female,No,Yes,No,0,No,No phone service,...,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,56.05,0,NaN
4331,Los Angeles,34.089953,-118.294824,Male,No,Yes,Yes,0,Yes,No,...,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,19.85,0,NaN
4687,Sun City,33.739412,-117.173334,Male,No,Yes,Yes,0,Yes,Yes,...,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,25.35,0,NaN
5104,Ben Lomond,37.078873,-122.090386,Female,No,Yes,Yes,0,Yes,No,...,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,20.00,0,NaN
5719,La Verne,34.144703,-117.770299,Male,No,Yes,Yes,0,Yes,No,...,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,19.70,0,NaN
6772,Bell,33.970343,-118.171368,Female,No,Yes,Yes,0,Yes,Yes,...,Yes,Yes,No,Two year,No,Mailed check,73.35,73.35,0,NaN


Tratamento na coluna Total Charges aplicado com sucesso.

In [15]:
data.to_excel(f'{datapath}/d02_intermediate/telco_customer_churn_v2.xlsx', index=False)

Arquivo salvo na pasta intermediária para uso da etapa de exploração do dado.